In [1]:
import pickle
import json
import gzip
import os
import subprocess
import numpy as np
import multiprocessing
import re 
import csv
import torch
import sys
import shutil
import random

import uuid
import datetime
import time

import bz2
import pandas as pd
# import dbmanager  as dbmanager
from os.path import join


# from nltk.tokenize import RegexpTokenizer
# import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
# nltk.download('punkt') # for english sentences tokenization

# tokenizer = RegexpTokenizer(r'\w+')

In [2]:
## Load non processed data
# file = './data/tvqa_qa_release/tvqa_train.jsonl'
# with open(file, 'r') as f:
#     lines = []
#     for l in f.readlines():
#         loaded_l = json.loads(l.strip("\n"))
#         lines.append(loaded_l)

In [3]:
def answers_to_trec(answers):
    trec_answers = {}
    key = 0
    for answer in answers:
        print(key, '_', answer)
#         print(answer)
        doc = '<DOC>\n' + \
            '<DOCNO>' + str(key) + '</DOCNO>\n' + \
            '<TITLE>' + answer + '</TITLE>\n' + \
            '</DOC>\n'
        trec_answers[str(key)] = doc
        key += 1
    return trec_answers

In [4]:
# def generate_index_dir():

# try:
#                 golden_file = sys.argv[1]
#                 predictions_file = sys.argv[2]
#         except:
#                 sys.exit("Provide golden and predictions files.")
        
#         try:
#                 system_name = sys.argv[3]
#         except :
#                 try:
#                         system_name = predictions_file.split('/')[-1]
#                 except:
#                         system_name = predictions_file

#         with open(golden_file, 'r') as f:
#                 golden_data = json.load(f)

#         with open(predictions_file, 'r') as f:
#                 predictions_data = json.load(f)

#         temp_dir = uuid.uuid4().hex
#         qrels_temp_file = '{0}/{1}'.format(temp_dir, 'qrels.txt')
#         qret_temp_file = '{0}/{1}'.format(temp_dir, 'qret.txt')

#         try:
#                 if not os.path.exists(temp_dir):
#                         os.makedirs(temp_dir)
#                 else:
#                         sys.exit("Possible uuid collision")

#                 format_bioasq2treceval_qrels(golden_data, qrels_temp_file)
#                 format_bioasq2treceval_qret(predictions_data, system_name, qret_temp_file)

#                 trec_evaluate(qrels_temp_file, qret_temp_file)
#         finally:
#                 os.remove(qrels_temp_file)
#                 os.remove(qret_temp_file)
#                 os.rmdir(temp_dir)

In [5]:
def to_trecfile(docs, filename, compression = 'yes'):
    # Pickle to Trectext converter
    doc_list = []
    if compression == 'yes':
        with gzip.open(filename,'wt') as f_out:
            docus = {}
            for key, value in docs.items():
                f_out.write(value)
    else:
        with open(filename,'wt') as f_out:
            docus = {}
            for key, value in docs.items():
                f_out.write(value)

In [6]:
def build_index(index_input, index_loc):
    if build_index_flag == 'no':
        return
# Build corpus index 
    if os.path.exists(index_loc):
        shutil.rmtree(index_loc)
        os.makedirs(index_loc)
    else:
        os.makedirs(index_loc) 
#     index_loc_param = '--indexPath=' + index_loc

    anserini_index = anserini_loc + 'target/appassembler/bin/IndexCollection'
    anserini_parameters = [
#                            'nohup', 
                           'sh',
                           anserini_index,
                           '-collection',
                           'TrecCollection',
                           '-generator',
                           'JsoupGenerator',
                           '-threads',
                            '16',
                            '-input',
                           index_input,
                           '-index',
                           index_loc,
                           '-storePositions',
                            '-keepStopwords',
                            '-storeDocvectors',
                            '-storeRawDocs']
#                           ' >& ',
#                           log_file,
#                            '&']



#     anserini_parameters = ['ls',
#                           index_loc]


    print(anserini_parameters)

    index_proc = subprocess.Popen(anserini_parameters,
            stdout=subprocess.PIPE, shell=False)
    (out, err) = index_proc.communicate()
    print(out.decode("utf-8"))
    print('Index error: ', err)


In [7]:
def query_to_trec(q_id, query, filename):
    q_t = {}
    q_t[q_id] = '<top>\n\n' + \
        '<num> Number: ' + str(q_id) + '\n' + \
        '<title> ' + query + '\n\n' + \
        '<desc> Description:' + '\n\n' + \
        '<narr> Narrative:' + '\n\n' + \
        '</top>\n\n'
    return q_t
    
#     queries_list = []
#     queries_dict = {}
#     query = {}
#     id_num = 0
#     ids_dict = {}
#     q_trec = {}
#     for query in q_dup_pos:
#         str_id = str(id_num)
#         id_new = str_id.rjust(15, '0')
        
#         key = query['doc_id']
#         q = questions[key]
# #         print(key)
#         text = remove_sc(q['title'] + ' ' + q['text']) #Join title and text 
#         query['number'] = key
# #         query['text'] = '#stopword(' + text + ')'
#         query['text'] = '(' + text + ')'
#         queries_list.append(dict(query))
        
#         q_t = '<top>\n\n' +           '<num> Number: ' + id_new + '\n' +           '<title> ' + text + '\n\n' +           '<desc> Description:' + '\n\n' +           '<narr> Narrative:' + '\n\n' +           '</top>\n\n'
#         q_trec[key] = q_t
# #         print(q)
#         ids_dict[str(id_num)] = key
#         id_num += 1
        
#     queries_dict['queries'] = queries_list
#     # with open(filename, 'wt', encoding='utf-8') as q_file:
#     with open(filename, 'wt') as q_file: #encoding option not working on python 2.7
#         json.dump(queries_dict, q_file, indent = 4)
        
#     return [q_trec, ids_dict]
        
#         ########################
#         ########################


In [8]:
def retrieve_docs(q_topics_file, retrieved_docs_file, index_loc, hits, b=0.2, k=0.8, N=10, M=10, Lambda=0.5):
    print(q_topics_file)
    #print(hits)
    anserini_search = anserini_loc + 'target/appassembler/bin/SearchCollection'
    command = [ 
               'sh',
               anserini_search,
               '-topicreader',
                'Trec',
                '-index',
                index_loc,
                '-topics',
                q_topics_file,
                '-output',
                retrieved_docs_file,
                '-bm25',
                '-b',
                str(b),
                '-k1',
                str(k),
                '-rm3',
                '-rm3.fbDocs',
                str(N),
                '-rm3.fbTerms',
                str(M),
                '-rm3.originalQueryWeight',
                str(Lambda),
                '-hits',
                str(hits), 
                '-threads',
                '10'
               ]
    print(command)
#     command = command.encode('utf-8')
    anserini_exec = subprocess.Popen(command, stdout=subprocess.PIPE, shell=False)
    (out, err) = anserini_exec.communicate()
#     print(out)
    print('Searching error: ', err)


In [9]:
def generate_preds_file(retrieved_docs_file):
    
    with open(retrieved_docs_file, 'rt') as f_in:
        try: 
            for doc in f_in:
                print(doc)
                q_id = doc.split(' ')[0]
                print(doc.split(' ')[2])
                pred_ans_id = doc.split(' ')[2]

            return pred_ans_id
        except:
            pred_ans_id = int(6) # When BM25+RM3 does not find any document, return an answer index outside the valid answer range
            return pred_ans_id
    
    
#             current_key = ids_dict[id_aux]
#             key_pair = current_key + '_' + doc.split(' ')[2]
#             all_dict[key_pair] = doc.split(' ')[4]
#         bm25_scores = [] 
#         i = 0
#         for query_dict in q_all:
#             i += 1
#             key_pair = query_dict['doc_id'] + '_' + query_dict['dup_id']
#             try: 
#                 query_dict['score'] = all_dict[key_pair]
#             except:
#                 query_dict['score'] = 0
# #            if i % 10000 == 0:
# #                 print('processed: ', i)
#             bm25_scores.append(dict(query_dict))
#         return bm25_scores

In [10]:
def baseline_compute(q_data, b,k,N,M,Lambda):
    query_id = q_data[0]
    query = q_data[1] # either question, subtitle, retrieval model?
    answers = q_data[2] # answers, always the same set
    print(query_id)
    
    temp_dir = workdir + str(query_id) + '_temp'  + '/'
    temp_index_input_dir = temp_dir + 'input_to_index/'
    temp_index_dir = temp_dir + 'index/'
    temp_index_input_file = temp_index_input_dir + 'trec_doc_input_file'
    temp_trec_query_file = temp_dir + 'trec_query_file'
    temp_retrieved_doc_file = temp_dir + 'retrieved_doc_file'
    
    
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        os.makedirs(temp_dir)
        os.makedirs(temp_index_input_dir)
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
        os.makedirs(temp_index_input_dir)
    
    # generate index input file
    trec_answers = answers_to_trec(answers)
    to_trecfile(trec_answers, temp_index_input_file, compression = 'no')
    
    # build index
    build_index(temp_index_input_dir, temp_index_dir)
    
    # generate query file
    trec_query = query_to_trec(query_id, query, temp_trec_query_file)
    to_trecfile(trec_query, temp_trec_query_file, compression = 'no')
    
    # get baseline scores file
    # hits = 1, because we are interested in the closest answer, nothing else
    retrieve_docs(temp_trec_query_file, temp_retrieved_doc_file, temp_index_dir, hits, b, k, N, M, Lambda)
    
    # Generate predictions
    predicted_answer_id = generate_preds_file(temp_retrieved_doc_file)

    # 
####     shutil.rmtree(temp_index_input_dir)
#     shutil.rmtree(temp_dir)
    
    return predicted_answer_id

In [11]:
def evaluate(predicted_answers, gold_answers):
    preds = np.asarray(predicted_answers)
    targets = np.asarray(gold_answers)
    acc = sum(preds == targets) / float(len(preds))
    return acc

In [12]:
def evaluate_params(params):
    b = params[0]
    k = params[1]
    N = params[2]
    M = params[3]
    Lambda = params[4]
    
    pred_answers = []
    gold_answers = []
    for item in questions_data:
        answers = [item['a' + str(i)] for i in range(0,5)]
        if model_type == 'qa':
            q_data = [item['qid'], item['q'], answers]
        elif model_type == 'sa':
            q_data = [item['qid'], item['q'], answers]
        elif model_type == 'retrieval':
            print('Retrieval model to be built...')
#             q_data = [item['qid'], item['q'], answers]

        predicted_answer_id = baseline_compute(q_data,b,k,N,M,Lambda)
        pred_answers.append(predicted_answer_id)
        gold_answers.append(item['answer_idx'])
    
    acc = evaluate(predicted_answers, gold_answers)
    results = [
        b,
        k,
        N,
        M,
        Lambda,
        float(acc)
    ]
    
    return results

In [13]:
# hits = 1
# params = [1,1,5,1,1]
# evaluate_params(params)


In [14]:
def start_process():
    print( 'Starting', multiprocessing.current_process().name)

In [15]:
def get_random_params(hyper_params, num_iter):
    random_h_params_list = []
    while len(random_h_params_list) < num_iter:
        random_h_params_set = []
        for h_param_list in hyper_params:
            sampled_h_param = random.sample(list(h_param_list), k=1)
#             print(type(sampled_h_param[0]))
#             print(sampled_h_param[0])
            random_h_params_set.append(round(sampled_h_param[0], 3))
        if not random_h_params_set in random_h_params_list:
            random_h_params_list.append(random_h_params_set)
#             print('Non repeated')
        else:
            print('repeated')
    return random_h_params_list

In [16]:
def find_best_dev_model(best_model_params_file, n_rand_iter, pool_size):
#     random_search = 'yes'
    
    if random_search == 'yes':
        ## Heavy random search
        brange = np.arange(0.1,1,0.05)
        krange = np.arange(0.1,4,0.1)
        N_range = np.arange(5,500,1) # num of docs
        M_range = np.arange(5,500,1) # num of terms
        lamb_range = np.arange(0,1,0.1) # weights of original query

        ## Light random search
#         brange = [0.2]
#         krange = [0.8]
#         N_range = np.arange(1,50,2)
#         M_range = np.arange(1,50,2)
#         lamb_range = np.arange(0,1,0.2)
        
        h_param_ranges = [brange, krange, N_range, M_range, lamb_range]
        params = get_random_params(h_param_ranges, n_rand_iter)

    else:
        brange = [0.2]
        krange = [0.8]
        N_range = [11]
        M_range = [10]
        lamb_range = [0.5]
       
        params = [[round(b,3), round(k,3), round(N,3), round(M,3), round(Lambda,3)] 
                  for b in brange for k in krange for N in N_range for M in M_range for Lambda in lamb_range]
    
#     print(len(params))
    pool = multiprocessing.Pool(processes=pool_size,
                                initializer=start_process,
                                )

#     pool_outputs = pool.map(baseline_computing, params)
    

    pool_outputs = pool.map_async(evaluate_params, params)
    print(pool_outputs.get())
    ###

    
    ##
    
    
    pool.close() # no more tasks
    while (True):
        if (pool_outputs.ready()): break
        remaining = pool_outputs._number_left
#         remaining2 = remaining1
#         remaining1 = pool_outputs._number_left
        if remaining%10 == 0:
            print("Waiting for", remaining, "tasks to complete...")
            time.sleep(2)
        
      
    pool.join()  # wrap up current tasks
    pool_outputs.get()
    params_file = './baselines/best_ir_model/' + 'tvqa' + '_' + 'bm25_rm3_' + data_split + '_hparams.pickle'
    pickle.dump(pool_outputs.get(), open(params_file, "wb" ) )
    print('Total parameters tested: ' + str(len(pool_outputs.get())))
    best_model_params = max(pool_outputs.get(), key=lambda x: x[5])
    
    best_model_dict = {
        'b': best_model_params[0],
        'k': best_model_params[1],
        'N': best_model_params[2],
        'M': best_model_params[3],
        'Lambda': best_model_params[4],
        'n_rand_iter': n_rand_iter,
        'hits': hits,
        'auc05_score': best_model_params[5],
    }
    best_model_dict = {k:str(v) for k, v in best_model_dict.items()} # everything to string
    
    print(best_model_dict)
    with open(best_model_params_file, 'wt') as best_model_f:
        json.dump(best_model_dict, best_model_f)

In [17]:
# def qa_prediction(data):
#     gold_answers = []
#     predicted_answers = []
#     print(len(data))
#     for item in data:
#         tfidf_vectorizer = TfidfVectorizer()
#         q = [item['q']]
#         answers = [item['a' + str(i)] for i in range(0,5)]
#     #     print(answers)
#         tfidf_q = tfidf_vectorizer.fit_transform(q)
#     #     print(tfidf_q)
#         tfidf_answers = tfidf_vectorizer.transform(answers)
#     #     print(tfidf_answers)
#         cosine_similarities = linear_kernel(tfidf_q, tfidf_answers).flatten()
#         related_docs_indices = cosine_similarities.argsort()[:-5:-1]
#         gold_answers.append(item['answer_idx'])
#         predicted_answers.append(related_docs_indices[0])
#     return [predicted_answers, gold_answers]

In [18]:
# def sa_prediction(data):
#     tfidf_vectorizer = TfidfVectorizer()
#     gold_answers = []
#     predicted_answers = []
#     for item in data:
#     #     print(item['q'])
#         sub = [item['located_sub_text']]
#         answers = [item['a' + str(i)] for i in range(0,5)]
#     #     print(answers)
#         tfidf_sub = tfidf_vectorizer.fit_transform(sub)
#     #     print(tfidf_q)
#         tfidf_answers = tfidf_vectorizer.transform(answers)
#     #     print(tfidf_answers)
#         cosine_similarities = linear_kernel(tfidf_sub, tfidf_answers).flatten()
#         related_docs_indices = cosine_similarities.argsort()[:-5:-1]
#         gold_answers.append(item['answer_idx'])
#         predicted_answers.append(related_docs_indices[0])
#     return [predicted_answers, gold_answers]

In [19]:
# [predicted_answers, gold_answers] = sa_prediction(processed_data_val)
# preds = np.asarray(predicted_answers)
# targets = np.asarray(gold_answers)
# acc = sum(preds == targets) / float(len(preds))
# acc

In [20]:
def retrieval_model(val_data, train_data):
    tfidf_vectorizer_q_train = TfidfVectorizer()
    gold_answers = []
    predicted_answers = []
    questions_train = [ele['q'] for ele in train_data]
    tfidf_q_train = tfidf_vectorizer_q_train.fit_transform(questions_train)
#     print(tfidf_q_train)
    j = 0
    for item in val_data:
        j += 1
#         print(j)
    #     print(item['q'])
#         print('main: ', tfidf_q_train.shape)
        try:
            q = [item['q']]
    #         q_train = [q for q in train_data['q']]
    #         answers = [item['a' + str(i)] for i in range(0,5)]
        #     print(answers)
    #         print(q)
            tfidf_q = tfidf_vectorizer_q_train.transform(q)
        #     print(tfidf_q)

        #     print(tfidf_answers)
            cosine_similarities_q_train = linear_kernel(tfidf_q, tfidf_q_train).flatten()
            related_docs_indices_q_train = cosine_similarities_q_train.argsort()[:-5:-1]
            q_similar_idx = related_docs_indices_q_train[0]
    #         print(q_similar)
            gold_a_idx = train_data[q_similar_idx]['answer_idx']
            gold_train_answer = [train_data[q_similar_idx]['a' + str(gold_a_idx)]]
    #         print(gold_train_answer)

            tfidf_vectorizer_val = TfidfVectorizer()
    #         print(gold_train_answer)
            tfidf_q_val = tfidf_vectorizer_val.fit_transform(gold_train_answer)
    #         print('second: ', tfidf_q_val.shape)

            answers = [item['a' + str(i)] for i in range(0,5)]
            tfidf_answers = tfidf_vectorizer_val.transform(answers)
            cosine_similarities = linear_kernel(tfidf_q_val, tfidf_answers).flatten()
            related_docs_indices = cosine_similarities.argsort()[:-5:-1]
            gold_answers.append(item['answer_idx'])
            predicted_answers.append(related_docs_indices[0])
        
        except: 
            print(train_data[q_similar_idx])
        
        if j%1000 == 0:
            print('processed: ', j)
#         gold_answers.append(item['answer_idx'])
#         predicted_answers.append(related_docs_indices[0])
    return [predicted_answers, gold_answers]

In [21]:
if __name__ == '__main__':
    train_file = './data/tvqa_train_processed.json'
    val_file = './data/tvqa_val_processed.json'
    build_index_flag = 'yes'
    random_search = 'yes'
    
    hits = 1
    
    anserini_loc = '../anserini/'
    n_rand_iter = 5
    pool_size = 1
    
    with open(val_file, 'r') as f:
        processed_data_val = json.load(f)

    with open(train_file, 'r') as f:
        processed_data_train = json.load(f)
        
    # Options
    mode = 'val'
    model_type = 'qa'
#     model_type = 'sa'
#     model_type = 'retrieval'
    
    workdir = './workdir/'
    if not os.path.exists(workdir):
        os.makedirs(workdir)
    
    if mode == 'val':
        print('Validation mode: ')
        questions_data = processed_data_val[0:100]
    elif mode == 'test':
        questions_data = processed_data_train
 

    best_model_params_file = './baselines/best_ir_model/tvqa_bm25_rm3_best_model_dev.json'
    
    find_best_dev_model(best_model_params_file, n_rand_iter, pool_size)



Validation mode: 
('Starting', 'PoolWorker-1')
122039
(0, '_', u'Because Sheldon is being rude .')
(1, '_', u"Because he does n't like Sheldon .")
(2, '_', u'Because they are having an argument .')
(3, '_', u'Because Howard wanted to have a private meal with Raj .')
(4, '_', u"Because Sheldon wo n't loan him money for food .")
['sh', '../anserini/target/appassembler/bin/IndexCollection', '-collection', 'TrecCollection', '-generator', 'JsoupGenerator', '-threads', '16', '-input', './workdir/122039_temp/input_to_index/', '-index', './workdir/122039_temp/index/', '-storePositions', '-keepStopwords', '-storeDocvectors', '-storeRawDocs']
2019-05-08 12:44:54,883 INFO  [main] index.IndexCollection (IndexCollection.java:429) - DocumentCollection path: ./workdir/122039_temp/input_to_index/
2019-05-08 12:44:54,883 INFO  [main] index.IndexCollection (IndexCollection.java:430) - Index path: ./workdir/122039_temp/index/
2019-05-08 12:44:54,884 INFO  [main] index.IndexCollection (IndexCollection.jav

IOError: [Errno 2] No such file or directory: './workdir/122039_temp/retrieved_doc_file'

2019-05-08 12:44:57,540 INFO  [main] index.IndexCollection (IndexCollection.java:429) - DocumentCollection path: ./workdir/122039_temp/input_to_index/
2019-05-08 12:44:57,541 INFO  [main] index.IndexCollection (IndexCollection.java:430) - Index path: ./workdir/122039_temp/index/
2019-05-08 12:44:57,541 INFO  [main] index.IndexCollection (IndexCollection.java:431) - CollectionClass: TrecCollection
2019-05-08 12:44:57,541 INFO  [main] index.IndexCollection (IndexCollection.java:432) - Generator: JsoupGenerator
2019-05-08 12:44:57,541 INFO  [main] index.IndexCollection (IndexCollection.java:433) - Threads: 16
2019-05-08 12:44:57,541 INFO  [main] index.IndexCollection (IndexCollection.java:434) - Stemmer: porter
2019-05-08 12:44:57,541 INFO  [main] index.IndexCollection (IndexCollection.java:435) - Keep stopwords? true
2019-05-08 12:44:57,541 INFO  [main] index.IndexCollection (IndexCollection.java:436) - Store positions? true
2019-05-08 12:44:57,541 INFO  [main] index.IndexCollection (Ind

In [ ]:
# [predicted_answers, gold_answers] = retrieval_model(processed_data_val, processed_data_train)
# preds = np.asarray(predicted_answers)
# targets = np.asarray(gold_answers)
# acc = sum(preds == targets) / float(len(preds))
# acc

In [ ]:
processed_data_train[3000]['answer_idx']

In [ ]:
questions_train = [ele['q'] for ele in processed_data_train]
questions_val = [ele['q'] for ele in processed_data_val]

In [ ]:
questions_train[0:5]

In [ ]:
td = TfidfVectorizer()
td_2 = TfidfVectorizer()

In [ ]:
tfidf_q_train = td.fit_transform(questions_train)
tfidf_q_val = td_2.fit_transform(questions_val)

In [ ]:
tfidf_q_train

In [ ]:
tfidf_q_val

In [ ]:
type(td)

In [ ]:
len(processed_data_val)

In [ ]:
processed_data_val[0]

In [ ]:
len(processed_data_train)